# Examples


In [8]:
%reload_ext autoreload
%autoreload 2

import numpy as np

from src.bearing_calculations import solve_K_and_C

## Task 1

Calculate K and C matrixes for fluid bearing (journal/tilting pad)

### Friswell example 5.5


In [ ]:
# Given parameters
D = 100  # mm, diameter
L = 30  # mm, length
f = 525  # N, static load
c = 0.1  # mm, radial clearance
omega = 1500  # rpm, rotational speed
eta = 0.1  # Pa.s, viscosity

# Necessary unit conversions
D = D / 1000  # Convert mm to m
L = L / 1000  # Convert mm to m
c = c / 1000  # Convert mm to m
omega = omega * (2 * np.pi) / 60  # Convert rpm to rad/s

K, C = solve_K_and_C(D, omega, eta, L, f, c)

print()
print("Stiffness matrix K (N/mm):")
print(K)

print()
print("Damping matrix C (N/mm):")
print(C)

S_s: 1.009797638653862
Eccentricity (epsilon): 0.26629750900547666

Stiffness matrix K (N/mm):
[[ 7.36622541e+07  1.01549317e+08]
 [-1.44129519e+08  5.06993379e+07]]

Damping matrix C (N/mm):
[[1339457.13410624 -471170.58442419]
 [-471170.58442419 1788622.96467409]]


### Al-bender example 10.21
